In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
#s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-14/hr=16/part-00000-76298f84-5414-49c5-88f6-a981ff06d94d-c000.csv .

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)

date_day = "2020-03-14"
hr = '*'

rrm_local_path = "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)



# user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
# user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# # user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
# user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-14/hr=*/*.csv


In [2]:
rdd_rrm_local = spark.sparkContext.textFile(rrm_local_path)
rdd_rrm_local.take(3)

['site,ap,ap2,band,time_epoch,rssi',
 '00193f17-4e86-4761-a6a4-d529e9491ea2,5c5b357eeb23,5c5b35aed877,5,1584146558,-81.0',
 '00193f17-4e86-4761-a6a4-d529e9491ea2,5c5b35aed70f,5c5b35aed818,24,1584147465,-53.0']

In [3]:
# rdd_rrm_local.map(lambda line: (line.split(',')[0], line.split(',')[1])).collect()

In [4]:

df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(rrm_local_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

In [5]:
df_Schema

StructType(List(StructField(site,StringType,true),StructField(ap,StringType,true),StructField(ap2,StringType,true),StructField(band,IntegerType,true),StructField(time_epoch,IntegerType,true),StructField(rssi,DoubleType,true)))

In [6]:
df_scan.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|00193f17-4e86-476...|5c5b357eeb23|5c5b35aed877|   5|1584146558|-81.0|
|00193f17-4e86-476...|5c5b35aed70f|5c5b35aed818|  24|1584147465|-53.0|
|00193f17-4e86-476...|5c5b35aed8c2|5c5b35aed791|   5|1584147464|-65.0|
|00226d33-cd80-4b1...|5c5b35500fe1|5c5b355015db|  24|1584146463|-85.0|
|00272816-8dc2-4f1...|5c5b35ae6d5b|5c5b35ae6dce|  24|1584146263|-67.0|
|00272816-8dc2-4f1...|5c5b35ae6db5|5c5b35ae6d60|   5|1584147464|-67.0|
|00272816-8dc2-4f1...|5c5b35ae6e05|5c5b35ae6e46|   5|1584147462|-68.0|
|00272816-8dc2-4f1...|5c5b35ae6e5a|5c5b35ae6d47|  24|1584147460|-71.0|
|00272816-8dc2-4f1...|5c5b35ae7152|5c5b35ae6d83|  24|1584147462|-59.0|
|0036a1b9-254b-42d...|5c5b352e88e0|5c5b352e677a|  24|1584147474|-62.0|
|0036a1b9-254b-42d...|5c5b352e8903|        null|   5|1584147476|-62.0|
|00458